# Gestão de Unidade Curricular

## Problema

Criar classe que use DataFrame Pandas como estrutura de dados e implemente  funcionalidades (métodos) como:
* classificacoes_aluno()  
* inscrito()  
* pauta()
* qual_a_turma() 
* alunosTurma()
* estatisticas()
* histograma()


Considere os dados no ficheiro `PA.xls`

## Resolução

### imports


In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import json

## ver como é o ficheiro PA.xls

In [ ]:
file = "PA.xls"
df = pd.read_excel(file, index_col="NumMec")  # Num Mec como indíce
df.head()

### Definição da classe



In [ ]:
class UC:
    '''classe para gestão de uma cadeira usando uma DataFrame'''

    def calc_classific_final(df):
        return round(0.3 * df["ETP"] + 0.4 * df["EP"] + 0.1 * df["AC"] + 0.2 * df["TrabP"]).astype("int")

    #-----------------------------------------------------------------
    def __init__(self, uc, file):
        '''construtor. Recebe informação de ficheiro Excel'''

        self.__df = pd.read_excel(file, index_col="NumMec")  # Num Mec como indíce
        self.__df.CURSO = self.__df.CURSO.astype("category")
        self.__df.TP = self.__df.TP.astype("category")
        self.__df.P = self.__df.P.astype("category")

        self.__df = self.__df.drop("Unnamed: 0", axis=1)

        ## adicionar coluna com nota final
        self.__df["FINAL"] = UC.calc_classific_final(self.__df)

        self.__uc = uc

        # debug
        # print(self.__df.info())
        
    
    #-----------------------------------------------------------------
    def estatisticas(self):
        '''Devolve várias estatísticas para as várias componentes da avaliação da UC'''

        return(str(self.__df.describe()))
    
    #-----------------------------------------------------------------
    def classificacoes_aluno(self, num_mec):
        '''Devolve as avaliações de um aluno'''
        
        return self.__df.loc[num_mec][["ETP","AC","TrabP","EP"]]
        
    
    
    #-----------------------------------------------------------------
    def classificacoes_aluno_json(self, num_mec):
        ''' Devolve json para informação de um estudante'''

        df = self.__df.loc[[num_mec]][["ETP","AC","TrabP","EP"]]
        #print(df)
        
        # convert to dictionary
        dic = df.to_dict(orient='records')
        
        return json.dumps(dic[0])
        
    #-----------------------------------------------------------------
    def inscrito(self, num_mec):
        '''Devolve True se aluno faz parte do curso '''
        
        # obter  alunos
        alunos = set(self.__df.index)
        #print(alunos)
        
        # verificar se num_mec faz parte dos alunos
        faz_parte = num_mec in alunos
        
        return faz_parte
    
        # alternativa: usar .loc 
    
    #-----------------------------------------------------------------    
    def qual_a_turma(self, num_mec):
        '''devolve tuplo com turma TP e turma P'''
        
        assert self.inscrito(num_mec)  #"não inscrito"

        # debug - linha com a informação do aluno
        print(self.__df.loc[num_mec])

        # obter TP aluno 
        turma_tp = self.__df.loc[num_mec]["TP"]  # .loc selects rows based on a labeled index. 
        print(turma_tp)

        turma_p = self.__df.loc[num_mec]["P"]
        print(turma_p)
        
        return (str(turma_p), str(turma_tp))

    #-----------------------------------------------------------------
    def alunos_turma(self, turma):
        '''Devolve lista dataframe de alunos de uma turma'''

        if turma[0] =="P":
            res = self.__df[self.__df["P"] == turma].sort_index()
        elif turma[0] =="T":
            res = self.__df[self.__df["TP"] == turma].sort_index()
        else:
            res = "Não existe esse tipo de turmas"

        return res

    #-----------------------------------------------------------------
    def melhores_alunos(self, n):
        '''Devolve informação acerca dos n melhores estudantes'''

        dfaux = self.__df.copy()
        
        dfaux = dfaux.sort_values("FINAL", ascending = False)  # TODO: passar de EP para NOTA_FINAL
        
        return dfaux[:n+1]
    
    #-----------------------------------------------------------------
    def pauta(self):
        '''Devolve toda a informação ordenada por turma prática e curso'''

        # 1 sort
        # 2 convert to string (without index)
        return self.__df.sort_values(["P","CURSO"]).to_string(index=True)
    
    #-----------------------------------------------------------------
    def pauta_em_xml(self):
        '''Devolve toda a informação em formato XML'''
        
        colunas = list(self.__df.columns)  # lista com nomes das colunas
        # header XML
        string = "<?xml version='1.0'?>\n"
        
        # tag de início de TURMA
        string += "<TURMA>\n"
        
        # processar row by row
        for indice, linha in self.__df.iterrows():
            
            # abrir tag <ALUNO>  e usar Num Mec com parametro (que se encontra no indíce)
            string += f"\t<ALUNO NumMec=\"{indice}\">\n"  # num mec
            
            # processar coluna a coluna
            for coluna in colunas[1:]:
                string += f"\t\t<{coluna}>{linha[coluna]}</{coluna}>\n"
            
            # terminar <ALUNO>
            string += f"\t</ALUNO>\n"
            
        # terminar turma
        string += "</TURMA>\n"
        
        return string
    
    #-----------------------------------------------------------------
    def histograma(self, componente ="FINAL"):
        '''Desenha o histograma da componente de avaliação, por defeito a FINAL'''

        sns.histplot(data=self.__df, 
                    x=componente,
                    kde = True, 
                    binrange=[0,21],
                    binwidth=1,
                    
                )
        



    #-----------------------------------------------------------------
    def __str__(self):
        '''Devolve string com dataframe'''
        
        return(str(self.__df))


    #------------ para debug
    #def get_df(self):
    #    return self.__df

## Testes à classe

### Criar objeto com dados do ficheiro PA.xls

In [ ]:
pa  = UC (uc="PA",file = "PA.xls" )  
print(pa)  # uses __str__()

### Operações relativas a alunos individuais

#### Verificar se aluno está inscrito na UC

In [ ]:

num = 19911

print()
if pa.inscrito(num):
    print(f"{num} está inscrito na UC")
    #pa.notas_estudante(num)
    #pa.nota_final_estudante(num)
else:
    print(f"{num} não consta dos inscritos na UC")
    
num = 12
if pa.inscrito(num):
    print(f"{num} está inscrito na UC")
else:
    print(f"{num} não consta dos inscritos na UC")

#### Determinar turmas  de um aluno

In [ ]:
num = 19911
pa.qual_a_turma(num)


#### Obter classificações para aluno (dado num mec)

In [ ]:
num = 19911
classifics = pa.classificacoes_aluno(num)
print(type(classifics))
print(classifics)

##### Classificações por aluno (em JSON)

In [ ]:
num = 19911
classifics = pa.classificacoes_aluno_json(num)
#print(type(classifics))
print(classifics)

### Operações relativas a turmas

#### Informação de alunos de uma turma (ordenado num mec)

In [ ]:
pa.alunos_turma("P1")

In [ ]:
pa.alunos_turma("TP1")

### Estatísticas

#### Histogramas 


In [ ]:
pa.histograma("ETP")

In [ ]:
pa.histograma("FINAL")

#### Estatísticas gerais da UC (nas várias componentes)


In [ ]:
print(pa.estatisticas())

### Pautas

#### Pauta


In [ ]:
pauta_pa = pa.pauta()

with open("turma.txt","w") as fileout:
    fileout.write(pauta_pa)


##### Pauta em XML

In [ ]:
r = pa.pauta_em_xml()
print(r)

with open("turma.xml","w") as fileout:
    fileout.write(r)

### Outras operações

#### Melhores 5 alunos 


In [ ]:
pa.melhores_alunos(5)   

# 